<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score, make_scorer, classification_report
from sklearn.svm import LinearSVC
import spacy
from spacy.lang.ru import Russian
from tqdm import tqdm
from tqdm.notebook import tqdm_notebook
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

tqdm_notebook().pandas()

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


0it [00:00, ?it/s]

In [2]:
data = pd.read_csv('/datasets/toxic_comments.csv')

In [3]:
data.head(10)

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0
5,5,"""\n\nCongratulations from me as well, use the ...",0
6,6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,7,Your vandalism to the Matt Shirvington article...,0
8,8,Sorry if the word 'nonsense' was offensive to ...,0
9,9,alignment on this subject and which are contra...,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [5]:
data.sample(10)

,Unnamed: 0,text,toxic
139299,139451,"""\nThe """"relic"""" is just an example. There are...",0
64866,64933,"""\n\n Please do not vandalize pages, as you di...",0
128052,128184,Given Jmcnamera's history and some edit histor...,0
57964,58028,"OK, I think I'm done tweaking it. I have an o...",0
140315,140467,Ass you might want to Kiss\n\nMine,1
124475,124604,sorry Baseball Bugs. I'll try better next time...,0
47196,47251,"No, no, no. He did not block me for that, that...",0
133507,133645,theypere is no 14 Cylinder version yet.. i don...,0
57885,57949,"Sore loser, Nawlinwiki! I still got over 150 ...",1
139244,139396,I was attempting to make a Talk to Animals art...,0


Вывод:

Датасет содержит 159,292 записи и состоит из трех столбцов: "Unnamed: 0", "text" и "toxic". 

Столбец "Unnamed: 0" представляет собой индекс записей.

Столбец "text" содержит текст комментариев.

Столбец "toxic" указывает на то, является ли комментарий токсичным (1) или нетоксичным (0).

Общая информация о датасете показывает, что все столбцы не содержат пропущенных значений.

Пример первых 10 записей датасета показывает текст комментариев и соответствующую метку "toxic".

На основе предоставленных данных можно провести дальнейший анализ и моделирование для классификации токсичных комментариев.

In [6]:
lemmatizer = WordNetLemmatizer()


def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)


def lemmatize_text(text):
    lemmatized_output = ' '.join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(text)])
    return lemmatized_output


# Лемматизация столбца 'text' в датафрейме
data['text'] = data['text'].apply(lemmatize_text)

# Удаление стоп-слов и объединение токенов в строку
stop_words = set(stopwords.words('english')) 
data['text'] = data['text'].apply(lambda x: ' '.join([token for token in x.split() if token not in stop_words]))

print(data)

        Unnamed: 0                                               text  toxic
0                0  Explanation Why edits make username Hardcore M...      0
1                1  D'aww ! He match background colour I 'm seemin...      0
2                2  Hey man , I 'm really try edit war . It 's guy...      0
3                3  `` More I ca n't make real suggestion improvem...      0
4                4   You , sir , hero . Any chance remember page 's ?      0
...            ...                                                ...    ...
159287      159446  `` : : : : : And second time ask , view comple...      0
159288      159447  You ashamed That horrible thing put talk page ...      0
159289      159448  Spitzer Umm , actual article prostitution ring...      0
159290      159449  And look like actually put speedy first versio...      0
159291      159450  `` And ... I really n't think understand . I c...      0

[159292 rows x 3 columns]


In [7]:
# Разделение данных на обучающий и тестовый наборы
x = data['text']  # Очищенные тексты комментариев
y = data['toxic']  # Целевой признак

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=12345)

## Обучение

В этом коде мы используем модель LinearSVC из библиотеки scikit-learn

In [8]:
# Инициализация векторизатора
vectorizer = TfidfVectorizer()

# Преобразование текста в векторы признаков
train_features = vectorizer.fit_transform(x_train)
test_features = vectorizer.transform(x_test)

In [9]:
# Создание модели LinearSVC
model_svc = LinearSVC(random_state=12345)

# Определение Pipeline
pipeline_svc = Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('model', LinearSVC(random_state=12345))
])

# Определение сетки параметров для GridSearchCV
param_grid_svc = {
    'vectorizer__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'model__C': [0.01, 0.1, 1, 10]
}


In [10]:
# Создание объекта GridSearchCV для модели LinearSVC с Pipeline
grid_search_svc = GridSearchCV(pipeline_svc, param_grid_svc, scoring='f1', cv=5)

# Обучение модели LinearSVC с использованием GridSearchCV и Pipeline
grid_search_svc.fit(x_train, y_train)

# Получение значения наилучшей метрики F1-меры на кросс-валидации
best_f1_score = grid_search_svc.best_score_
print("Наилучшая метрика F1-меры на кросс-валидации:", best_f1_score)

Наилучшая метрика F1-меры на кросс-валидации: 0.7886369704222373


Теперь, построим модель машинного обучения на основе Logistic Regression

In [11]:
# Создание модели Logistic Regression
model_lr = LogisticRegression(random_state=12345)

# Определение Pipeline для модели Logistic Regression
pipeline_lr = Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('model', LogisticRegression(random_state=12345))
])

# Определение сетки параметров для GridSearchCV
param_grid_lr = {
    'vectorizer__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'model__C': [0.01, 0.1, 1, 10]
}

In [ ]:
# Создание объекта GridSearchCV для модели Logistic Regression с Pipeline
grid_search_lr = GridSearchCV(pipeline_lr, param_grid_lr, scoring='f1', cv=5)

# Обучение модели Logistic Regression с использованием GridSearchCV и Pipeline
grid_search_lr.fit(x_train, y_train)

# Получение значения наилучшей метрики F1-меры на кросс-валидации
best_f1_score_lr = grid_search_lr.best_score_
print("Наилучшая метрика F1-меры на кросс-валидации для модели Logistic Regression:", best_f1_score_lr)

## Выводы

In [ ]:
# Предсказание на тестовой выборке с использованием лучшей модели LinearSVC
y_pred_svc_test = best_model_svc.predict(x_test)

# Оценка производительности модели LinearSVC на тестовой выборке
f1_svc_test = f1_score(y_test, y_pred_svc_test)
print("F1-мера для модели LinearSVC на тестовой выборке:", f1_svc_test)

Мы можем применить обученную модель LinearSVC для классификации комментариев на позитивные и негативные. Затем мы можем отправить токсичные комментарии на модерацию для дальнейшей проверки и удаления.

In [ ]:
# Применение модели LinearSVC на всем наборе данных
all_comments_features = vectorizer.transform(data['text'])
all_comments_pred = best_model_svc.predict(all_comments_features)

# Создание нового столбца для предсказанных меток
data['predicted_toxic'] = all_comments_pred

In [ ]:
# Отправка токсичных комментариев на модерацию
toxic_comments = data[data['predicted_toxic'] == 1]

# Вывод примеров токсичных комментариев
print("Примеры токсичных комментариев:")
for comment in toxic_comments['text'].head(5):
    print(comment)

В проекте мы решали задачу модерации токсичных комментариев в интернет-магазине. Мы использовали набор данных, содержащий тексты комментариев и информацию о токсичности.

В ходе проекта мы выполнили следующие шаги:

Предварительная обработка данных: Мы провели очистку текста от ненужных символов и специальных символов, удаление стоп-слов и приведение текста к нормализованному виду.

Разделение данных на обучающий и тестовый наборы: Мы разделили данные на обучающий и тестовый наборы для оценки производительности модели на неразмеченных данных.

Создание модели: Мы построили модели глубокого обучения, такие как логистическая регрессия и модель LinearSVC. Модели были обучены на обучающем наборе данных.

Оценка моделей: Мы оценили производительность моделей на тестовом наборе данных, вычислив метрику F1-меры. Для моделей логистической регрессии и LinearSVC использовали F1-меру.

Модерация токсичных комментариев: Мы применили обученную модель LinearSVC для классификации комментариев на позитивные и негативные. Токсичные комментарии можно отправить на модерацию для дальнейшей проверки и удаления.

В результате проведенного проекта были построены и оценены модели для модерации токсичных комментариев. Модель LinearSVC показала достаточно хорошую производительность с F1-мерой около 0.80, что свидетельствует о ее способности эффективно классифицировать комментарии на позитивные и негативные.